# Linear To Cubic Process
This process will compute a set of support functions in the linear scaling mode, and then use those to fit the KS-orbitals of the cubic scaling mode.

In [ ]:
functionals = ["PBE", "HF", "B3LYP", "PBE0"]

markers = {}
msize = {}
markers["PBE"] = "+"
markers["BLYP"] = "x"
markers["HF"] = "d"
markers["B3LYP"] = "v"
markers["PBE0"] = "."

msize["PBE"] = 12
msize["BLYP"] = 12
msize["HF"] = 8
msize["B3LYP"] = 8
msize["PBE0"] = 12

Read in the input.

In [ ]:
from BigDFT.IO import read_pdb
from os.path import join

with open(join("input", "renumber.pdb")) as ifile:
    sys = read_pdb(ifile)

Parameters for two versions of the PBE run.

In [ ]:
from BigDFT.Calculators import SystemCalculator
code = SystemCalculator(skip=True, verbose=False)

In [ ]:
from BigDFT.Inputfiles import Inputfile
inp = Inputfile()
inp.set_xc("PBE")
inp.set_hgrid(0.37)
inp.update({'output': {'orbitals': 'text'}})
inp.set_psp_krack()

In [ ]:
lin_inp = Inputfile()
lin_inp.set_xc("PBE")
lin_inp.set_hgrid(0.37)
lin_inp["import"] = "linear"
lin_inp["lin_general"] = {'output_wf': 21}
lin_inp.set_psp_krack()

Run.

In [ ]:
log_cubic = code.run(sys=sys, input=inp, 
                     run_dir="work", name="cubic")

In [ ]:
log_linear = code.run(sys=sys, input=lin_inp, 
                      run_dir="work", name="linear")

## Orbital Processing
First we need to convert all of the cubic wavefunctions to cubefiles using BigDFTool.

In [ ]:
from CubeFiles import generate_cube

In [ ]:
for i in range(1, log_cubic.number_of_orbitals + 1):
    generate_cube(log_cubic, i, temp_dir="temp_cubic")

Read them in.

In [ ]:
from os.path import join
from CubeFiles import CubeFile, cube_name

cubic_cubes = {}
for i in range(1, log_cubic.number_of_orbitals + 1):
    cubic_cubes[i] = CubeFile()
    with open(join("temp_cubic", cube_name(i) + ".cube")) as ifile:
        cubic_cubes[i].read(ifile)
        
linear_cubes = {}
for i in range(1, log_linear.log["Total No. Support Functions"] + 1):
    linear_cubes[i] = CubeFile()
    with open(join(log_linear.srcdir, log_linear.data_directory, 
                   cube_name(i, wf_type="SupFun") + ".cube")) as ifile:
        linear_cubes[i].read(ifile)

Compute the overlap between the two representations.

In [ ]:
from numpy import zeros
ovlp = zeros((log_cubic.number_of_orbitals, 
              log_linear.log["Total No. Support Functions"]))
for i in range(0, ovlp.shape[0]):
    for j in range(0, ovlp.shape[1]):
        ovlp[i, j] = cubic_cubes[i+1].dot(linear_cubes[j+1])

Compute the overlap between just the linear.

In [ ]:
S = zeros((log_linear.log["Total No. Support Functions"],
           log_linear.log["Total No. Support Functions"]))
for i in range(0, S.shape[0]):
    for j in range(0, S.shape[1]):
        S[i, j] = linear_cubes[i+1].dot(linear_cubes[j+1])

We can compare this overlap matrix to the one computed in the wavelet basis by BigDFT as a sanity check.

In [ ]:
from scipy.io import mmread
Sanaly = mmread(join(log_linear.srcdir, log_linear.data_directory, "overlap_sparse.mtx"))

In [ ]:
from numpy.linalg import norm
print(norm(S - Sanaly.todense()))

Compute the coefficients and density matrix.

In [ ]:
from scipy.linalg import inv
Sinv = inv(S)

In [ ]:
from scipy.sparse import csr_matrix
K = {}
coef = ovlp.dot(Sinv).T
K["PBE"] = csr_matrix(2*coef.dot(coef.T))

In [ ]:
from numpy import trace
print(trace(K["PBE"].dot(Sanaly.todense())))

## Energy Comparison
Compute the energy difference between the two density matrices.

In [ ]:
from scipy.io import mmread
H = mmread(join(log_linear.srcdir, log_linear.data_directory, "hamiltonian_sparse.mtx"))
Kpbe_orig = mmread(join(log_linear.srcdir, log_linear.data_directory, "density_kernel_sparse.mtx"))

In [ ]:
from numpy import trace
from numpy.linalg import norm
err =  trace(H.dot(K["PBE"].todense())) - trace(H.dot(Kpbe_orig).todense())
print("Absolute", err)
print("Relative", 100*err/trace(H.dot(Kpbe_orig).todense()))

## PBE0 Calculation
Now perform a PBE0 calculation.

In [ ]:
inp = Inputfile()
inp.set_xc(-406)
inp.set_hgrid(0.37)
inp.update({'output': {'orbitals': 'text'}})
inp.set_psp_krack()

In [ ]:
log_pbe0 = code.run(sys=sys, input=inp, 
                    run_dir="work", name="pbe0")

Read orbitals as cube files.

In [ ]:
for i in range(1, log_pbe0.number_of_orbitals + 1):
    print(generate_cube(log_pbe0, i, temp_dir="temp_pbe0"))
pbe0_cubes = {}
for i in range(1, log_pbe0.number_of_orbitals + 1):
    pbe0_cubes[i] = CubeFile()
    with open(join("temp_pbe0", cube_name(i) + ".cube")) as ifile:
        pbe0_cubes[i].read(ifile)

Compute the density.

In [ ]:
ovlp = zeros((log_pbe0.number_of_orbitals, 
              log_linear.log["Total No. Support Functions"]))
for i in range(0, ovlp.shape[0]):
    for j in range(0, ovlp.shape[1]):
        ovlp[i, j] = pbe0_cubes[i+1].dot(linear_cubes[j+1])

In [ ]:
coef = ovlp.dot(Sinv).T
K["PBE0"] = csr_matrix(2*coef.dot(coef.T))

## Hartree-Fock Calculation

In [ ]:
inp = Inputfile()
inp.set_xc("HF")
inp.set_hgrid(0.37)
inp.update({'output': {'orbitals': 'text'}})
inp.set_psp_krack()

In [ ]:
log_hf = code.run(sys=sys, input=inp, 
                  run_dir="work", name="HF")

In [ ]:
for i in range(1, log_hf.number_of_orbitals + 1):
    print(generate_cube(log_hf, i, temp_dir="temp_hf"))
hf_cubes = {}
for i in range(1, log_hf.number_of_orbitals + 1):
    hf_cubes[i] = CubeFile()
    with open(join("temp_hf", cube_name(i) + ".cube")) as ifile:
        hf_cubes[i].read(ifile)

In [ ]:
ovlp = zeros((log_hf.number_of_orbitals, 
              log_linear.log["Total No. Support Functions"]))
for i in range(0, ovlp.shape[0]):
    for j in range(0, ovlp.shape[1]):
        ovlp[i, j] = hf_cubes[i+1].dot(linear_cubes[j+1])
coef = ovlp.dot(Sinv).T
K["HF"] = csr_matrix(2*coef.dot(coef.T))

## B3LYP Calculation

In [ ]:
inp = Inputfile()
inp.set_xc("-475")
inp.set_hgrid(0.37)
inp.update({'output': {'orbitals': 'text'}})
inp.set_psp_krack()

In [ ]:
log_b3lyp = code.run(sys=sys, input=inp, 
                     run_dir="work", name="B3LYP")

In [ ]:
for i in range(1, log_b3lyp.number_of_orbitals + 1):
    print(generate_cube(log_b3lyp, i, temp_dir="temp_b3lyp"))
b3lyp_cubes = {}
for i in range(1, log_b3lyp.number_of_orbitals + 1):
    b3lyp_cubes[i] = CubeFile()
    with open(join("temp_b3lyp", cube_name(i) + ".cube")) as ifile:
        b3lyp_cubes[i].read(ifile)

In [ ]:
ovlp = zeros((log_b3lyp.number_of_orbitals, 
              log_linear.log["Total No. Support Functions"]))
for i in range(0, ovlp.shape[0]):
    for j in range(0, ovlp.shape[1]):
        ovlp[i, j] = b3lyp_cubes[i+1].dot(linear_cubes[j+1])
coef = ovlp.dot(Sinv).T
K["B3LYP"] = csr_matrix(2*coef.dot(coef.T))

## QM-CR Quantities
Compute some QM-CR quantities.

In [ ]:
from BigDFT.PostProcessing import BigDFTool
tool = BigDFTool()

In [ ]:
pi_orig = tool.run_compute_purity(sys, log_linear, kxs=Kpbe_orig.dot(Sanaly))

pi = {}
for f in functionals:
    pi[f] = tool.run_compute_purity(sys, log_linear, kxs=K[f].dot(Sanaly))

In [ ]:
fbo_orig = tool.fragment_bond_order(sys, sys, sys, log_linear, kxs=Kpbe_orig.dot(Sanaly))

fbo = {}
for f in functionals:
    fbo[f] = tool.fragment_bond_order(sys, sys, sys, log_linear, kxs=K[f].dot(Sanaly))

In [ ]:
order = sorted(sys, key=lambda x: int(x.split(":")[1]))
relevant = []

for k1, v1 in fbo_orig.items():
    for k2, v2 in v1.items():
        if k1 == k2:
            continue
        if (k2, k1) in relevant:
            continue
        if "MOL" in k1:
            if v2 > 1e-2:
                relevant.append((k1, k2))
        else:
            if v2 > 5e-3:
                relevant.append((k1, k2))
relevant = sorted(relevant, key=lambda x: int(x[0].split(":")[1]))

Plot.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=(8, 5))

axs[0].plot([abs(pi_orig[x]) for x in sys], 
            linestyle='--', linewidth=2, color='k')
for f in functionals:
    axs[0].plot([(pi[f][x]) for x in sys], label=f,
                marker=markers[f], markersize=msize[f])
axs[0].set_xticks(range(len(order)))
axs[0].set_xticklabels(order, rotation=90)
axs[0].set_ylim(-1e-3, -5e-2)
axs[0].tick_params(axis='both', which='major', labelsize=16)
axs[0].set_ylabel("Purity Indicator", fontsize=18)

axs[1].plot([fbo_orig[x[0]][x[1]] for x in relevant], label="PBE-Linear", 
            linestyle='--', linewidth=2, color='k')
for f in functionals:
    axs[1].plot([fbo[f][x[0]][x[1]] for x in relevant], label=f, 
                marker=markers[f], markersize=msize[f])
axs[1].legend(loc="lower center", ncol=2, prop={'size': 11})
axs[1].set_ylim(5e-3, 1e-1)
axs[1].set_xticks(range(len(relevant)))
axs[1].set_xticklabels(["-".join(x) for x in relevant], rotation=90)
axs[1].set_yscale("log")
axs[1].tick_params(axis='both', which='major', labelsize=16)

axs[1].legend()

fig.tight_layout()